In [5]:
from pyspark.sql import SparkSession


spark = SparkSession.builder.getOrCreate()

spark.sql("show catalogs").show()

+-------------+
|      catalog|
+-------------+
|spark_catalog|
+-------------+



23/07/21 14:11:06 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 512123 ms exceeds timeout 120000 ms
23/07/21 14:11:06 WARN SparkContext: Killing executors is not supported by current scheduler.
23/07/21 14:11:11 WARN Executor: Issue communicating with driver in heartbeater
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:322)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEndpointRef.askSync(RpcEndpointRef.scala:101)
	at org.apache.spark.rpc.RpcEndpointRef.askSync(RpcEndpointRef.scala:85)
	at org.apache.spark.storage.BlockManagerMaster.registerBlockManager(BlockManagerMaster.scala:80)
	at org.apache.spark.storage.BlockManager.reregister(BlockManager.scala:641)
	at org.apache.spark.executor.Executor.reportHeartBeat(Executor.scala:1111)
	at org.apache.spark.executor.Executor.$anonfun$heartbeater$1(Executor.scala:244)
	at sc

In [3]:
from pyspark.sql import SparkSession
from pyspark.sql.types import IntegerType
from typedspark._core.column import Column
from typedspark._schema.schema import Schema
from typedspark._utils.create_dataset import create_empty_dataset


class A(Schema):
    id: Column[IntegerType]
    key: Column[IntegerType]
    value: Column[IntegerType]

spark = SparkSession.builder.getOrCreate()

df = create_empty_dataset(spark, A)

df.unpivot(A.id, A.key, A.value.str, "new_values")

TypeError: Column is not iterable